In [25]:
import pandas as pd
import numpy as np

In [7]:
CRAWL_EMBEDDING_PATH = '../input/crawl-300d-2M.vec'
GLOVE_EMBEDDING_PATH = '../input/glove.840B.300d.txt'

In [8]:
train = pd.read_hdf('../input/train.h5')
test = pd.read_hdf('../input/test.h5')

In [9]:
train.head()

,id,target,comment_text,severe_toxicity,obscene,identity_attack,insult,threat,asian,atheist,...,article_id,rating,funny,wow,sad,likes,disagree,sexual_explicit,identity_annotator_count,toxicity_annotator_count
0,59848,0.000000,"This is so cool. It's like, 'would you want yo...",0.000000,0.0,0.000000,0.00000,0.0,NaN,NaN,...,2006,rejected,0,0,0,0,0,0.0,0,4
1,59849,0.000000,Thank you!! This would make my life a lot less...,0.000000,0.0,0.000000,0.00000,0.0,NaN,NaN,...,2006,rejected,0,0,0,0,0,0.0,0,4
2,59852,0.000000,This is such an urgent design problem; kudos t...,0.000000,0.0,0.000000,0.00000,0.0,NaN,NaN,...,2006,rejected,0,0,0,0,0,0.0,0,4
3,59855,0.000000,Is this something I'll be able to install on m...,0.000000,0.0,0.000000,0.00000,0.0,NaN,NaN,...,2006,rejected,0,0,0,0,0,0.0,0,4
4,59856,0.893617,haha you guys are a bunch of losers.,0.021277,0.0,0.021277,0.87234,0.0,0.0,0.0,...,2006,rejected,0,0,0,1,0,0.0,4,47


In [10]:
train.columns

Index(['id', 'target', 'comment_text', 'severe_toxicity', 'obscene',
       'identity_attack', 'insult', 'threat', 'asian', 'atheist', 'bisexual',
       'black', 'buddhist', 'christian', 'female', 'heterosexual', 'hindu',
       'homosexual_gay_or_lesbian', 'intellectual_or_learning_disability',
       'jewish', 'latino', 'male', 'muslim', 'other_disability',
       'other_gender', 'other_race_or_ethnicity', 'other_religion',
       'other_sexual_orientation', 'physical_disability',
       'psychiatric_or_mental_illness', 'transgender', 'white', 'created_date',
       'publication_id', 'parent_id', 'article_id', 'rating', 'funny', 'wow',
       'sad', 'likes', 'disagree', 'sexual_explicit',
       'identity_annotator_count', 'toxicity_annotator_count'],
      dtype='object')

In [24]:
train[train["severe_toxicity"] > 0.5]["comment_text"]

44188                  Kill all the nigggers and pedophiles!
1341396    BURN THIS DOG RAPING WHITE NIGER ALIVE\nTIME f...
1370031    The righteous ALT LEFT will eradicate fascist ...
1377740    Awesome! Lets cut the head off hate! Lets stab...
1443046    .\n.\nIs there really a God ?\n... I once thou...
1519955    We will send wave after wave of American soldi...
1532869    You are murdering feckless SCUM.  Your mother ...
1679044    DOGS----\nEAT\nSHIT\nSLEEP\nBARK\n\nNIGERS\nEA...
Name: comment_text, dtype: object

In [27]:
text = train["comment_text"]

In [32]:
text[text == "Take care not to believe your own bullshit, see On Bullshit."]

Series([], Name: comment_text, dtype: object)

In [33]:
ttext = test["comment_text"]

In [34]:
ttext[ttext == "Take care not to believe your own bullshit, see On Bullshit."]

Series([], Name: comment_text, dtype: object)

In [35]:
# do the embeddings have text in other languages?
text.head()

0    This is so cool. It's like, 'would you want yo...
1    Thank you!! This would make my life a lot less...
2    This is such an urgent design problem; kudos t...
3    Is this something I'll be able to install on m...
4                 haha you guys are a bunch of losers.
Name: comment_text, dtype: object